In [1]:
# importing the important libraries

In [7]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import folium # plotting library
from pandas.io.json import json_normalize # tranforming json file into a pandas dataframe library
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values
# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 


In [4]:
# important info for the API

In [5]:
CLIENT_ID = 'DMN1ABJNSVBWYCXMG2WFGSOZYP4HZLRBMNFBHXHN3IYUE3NJ' # my Foursquare ID
CLIENT_SECRET = 'IUXOE5DSTR3YSEGLWZOJQBGZ2NTQG2VHO1XQRYY2MHSTC5XW' # my Foursquare Secret
VERSION = '20180604'


In [9]:
# Getting the longitude and latitude oa New York
address = 'New York'
geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

40.7127281 -74.0060152


In [21]:
# function to get the data of new york
def get_new_york_data():
    url='https://cocl.us/new_york_dataset'
    results=requests.get(url).json()
    # all data is present in features label
    features=results['features']
    
    # define the dataframe columns
    column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 
    # making the dataframe
    new_york_data = pd.DataFrame(columns=column_names)
    
    for data in features:
        borough = data['properties']['borough'] 
        neighborhood_name = data['properties']['name']
        
        neighborhood_latlon = data['geometry']['coordinates']
        neighborhood_lat = neighborhood_latlon[1]
        neighborhood_lon = neighborhood_latlon[0]
    
        new_york_data = new_york_data.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)
    
    return new_york_data


In [24]:
new_york_data=get_new_york_data()
new_york_data

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585
...,...,...,...,...
301,Manhattan,Hudson Yards,40.756658,-74.000111
302,Queens,Hammels,40.587338,-73.805530
303,Queens,Bayswater,40.611322,-73.765968
304,Queens,Queensbridge,40.756091,-73.945631


In [30]:
# function to intract with FourSquare API and get top 100 venues
def get_venues(lat,lng):
    
    #set variables
    radius = 500
    LIMIT=100

    #url to fetch data from foursquare api
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng,
            radius, 
            LIMIT)
    
    # get all the data
    results = requests.get(url).json()
    venue_data=results["response"]['groups'][0]['items']
    venue_details=[]
    for row in venue_data:
        try:
            venue_id=row['venue']['id']
            venue_name=row['venue']['name']
            venue_category=row['venue']['categories'][0]['name']
            venue_details.append([venue_id,venue_name,venue_category])
        except KeyError:
            pass
        
    column_names=['ID','Name','Category']
    df = pd.DataFrame(venue_details,columns=column_names)
    return df


In [ ]:
# getting the Egyptian restaurant in New York
column_names=['Borough', 'Neighborhood', 'ID','Name']
egyptian=pd.DataFrame(columns=column_names)
count=1
for row in new_york_data.values.tolist():
    Borough, Neighborhood, Latitude, Longitude=row
    venues = get_venues(Latitude,Longitude)
    egyptian_resturants=venues[venues['Category']=='Egyptian Restaurant']   
    print('(',count,'/',len(new_york_data),')','Egyptian Resturants in '+Neighborhood+', '+Borough+':'+str(len(egyptian_resturants)))
    for resturant_detail in egyptian_resturants.values.tolist():
        id, name , category=resturant_detail
        egyptian = egyptian.append({'Borough': Borough,
                                                'Neighborhood': Neighborhood, 
                                                'ID': id,
                                                'Name' : name
                                               }, ignore_index=True)
    count+=1


In [32]:
egyptian

,Borough,Neighborhood,ID,Name
0,Manhattan,Little Italy,5d8e37f352f64b000873b779,Zooba
1,Manhattan,Soho,5d8e37f352f64b000873b779,Zooba
2,Queens,Ridgewood,550b13c6498e8b5069630cee,Little Egypt


In [ ]:
# Their are only 3 restaurants.

In [34]:
# gets the details of the venue
def get_venue_details(venue_id):
        
    url = 'https://api.foursquare.com/v2/venues/{}?&client_id={}&client_secret={}&v={}'.format(
            venue_id,
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION)
    
    # get all the data
    results = requests.get(url).json()
    venue_data=results['response']['venue']
    venue_details=[]
    try:
        venue_id=venue_data['id']
        venue_name=venue_data['name']
        venue_likes=venue_data['likes']['count']
        venue_rating=venue_data['rating']
        venue_tips=venue_data['tips']['count']
        venue_details.append([venue_id,venue_name,venue_likes,venue_rating,venue_tips])
    except KeyError:
        pass
        
    column_names=['ID','Name','Likes','Rating','Tips']
    df = pd.DataFrame(venue_details,columns=column_names)
    return df


In [38]:
# we can now get the likes, tips and rating of the 3 restaurants
column_names=['Borough', 'Neighborhood', 'ID','Name','Likes','Rating','Tips']
egyptian_stats=pd.DataFrame(columns=column_names)
count=1


for row in egyptian.values.tolist():
    Borough,Neighborhood,ID,Name=row
    try:
        venue_details=get_venue_details(ID)
        print(venue_details)
        id,name,likes,rating,tips=venue_details.values.tolist()[0]
    except IndexError:
        print('No data available for id=',ID)
        # we will assign 0 value for these resturants as they may have been 
        #recently opened or details does not exist in FourSquare Database
        id,name,likes,rating,tips=[0]*5
    print('(',count,'/',len(egyptian),')','processed')
    egyptian_stats = egyptian_stats.append({'Borough': Borough,
                                                'Neighborhood': Neighborhood, 
                                                'ID': id,
                                                'Name' : name,
                                                'Likes' : likes,
                                                'Rating' : rating,
                                                'Tips' : tips
                                               }, ignore_index=True)
    count+=1


                         ID   Name  Likes  Rating  Tips
0  5d8e37f352f64b000873b779  Zooba     52     8.4    16
( 1 / 3 ) processed
                         ID   Name  Likes  Rating  Tips
0  5d8e37f352f64b000873b779  Zooba     52     8.4    16
( 2 / 3 ) processed
                         ID          Name  Likes  Rating  Tips
0  550b13c6498e8b5069630cee  Little Egypt     11     8.1     4
( 3 / 3 ) processed


In [39]:
egyptian_stats

,Borough,Neighborhood,ID,Name,Likes,Rating,Tips
0,Manhattan,Little Italy,5d8e37f352f64b000873b779,Zooba,52,8.4,16
1,Manhattan,Soho,5d8e37f352f64b000873b779,Zooba,52,8.4,16
2,Queens,Ridgewood,550b13c6498e8b5069630cee,Little Egypt,11,8.1,4


In [51]:
# save it to reduce the API calls as it is limited.
egyptian_stats.to_csv('egyptian_stats.csv', index=False)

In [54]:
egyptian_stats = pd.read_csv('egyptian_stats.csv')
egyptian_stats

,Borough,Neighborhood,ID,Name,Likes,Rating,Tips
0,Manhattan,Little Italy,5d8e37f352f64b000873b779,Zooba,52,8.4,16
1,Manhattan,Soho,5d8e37f352f64b000873b779,Zooba,52,8.4,16
2,Queens,Ridgewood,550b13c6498e8b5069630cee,Little Egypt,11,8.1,4


In [82]:
# adding the location of each restaurant
def loca(address2):
  geolocator2 = Nominatim(user_agent="foursquare_agent")
  location2 = geolocator2.geocode(address2)
  latitude2 = location2.latitude
  longitude2 = location2.longitude
  print(latitude2, longitude2)

loca('Little Italy, manhattan, NY')
loca('Soho, manhattan, NY')
loca('Ridgewood, Queens, NY')



40.7192728 -73.9982152
40.72288 -73.9987505
40.68912345 -73.88646349029689


In [84]:
data = [[40.7192728, -73.9982152], [40.72288, -73.9987505], [40.68912345, -73.88646349029689]] 
df = pd.DataFrame(data, columns=["Latitude", "Longitude"])
df

,Latitude,Longitude
0,40.719273,-73.998215
1,40.722880,-73.998750
2,40.689123,-73.886463


In [87]:
egyptian_stats2 = egyptian_stats.assign(Latitude = df["Latitude"], Longitude = df["Longitude"])
egyptian_stats2

,Borough,Neighborhood,ID,Name,Likes,Rating,Tips,Latitude,Longitude
0,Manhattan,Little Italy,5d8e37f352f64b000873b779,Zooba,52,8.4,16,40.719273,-73.998215
1,Manhattan,Soho,5d8e37f352f64b000873b779,Zooba,52,8.4,16,40.722880,-73.998750
2,Queens,Ridgewood,550b13c6498e8b5069630cee,Little Egypt,11,8.1,4,40.689123,-73.886463


In [55]:
# the highest num of likes
egyptian_stats2.iloc[egyptian_stats['Likes'].idxmax()]

Borough                        Manhattan
Neighborhood                Little Italy
ID              5d8e37f352f64b000873b779
Name                               Zooba
Likes                                 52
Rating                               8.4
Tips                                  16
Name: 0, dtype: object

In [60]:
# the lowest num of likes 
egyptian_stats2.iloc[egyptian_stats['Likes'].idxmin()]

Borough                           Queens
Neighborhood                   Ridgewood
ID              550b13c6498e8b5069630cee
Name                        Little Egypt
Likes                                 11
Rating                               8.1
Tips                                   4
Name: 2, dtype: object

In [61]:
# the highest num of tips
egyptian_stats2.iloc[egyptian_stats['Tips'].idxmax()]

Borough                        Manhattan
Neighborhood                Little Italy
ID              5d8e37f352f64b000873b779
Name                               Zooba
Likes                                 52
Rating                               8.4
Tips                                  16
Name: 0, dtype: object

In [62]:
# the lowest num of tips
egyptian_stats2.iloc[egyptian_stats['Tips'].idxmin()]

Borough                           Queens
Neighborhood                   Ridgewood
ID              550b13c6498e8b5069630cee
Name                        Little Egypt
Likes                                 11
Rating                               8.1
Tips                                   4
Name: 2, dtype: object

In [63]:
# the highest rating
egyptian_stats2.iloc[egyptian_stats['Rating'].idxmax()]

Borough                        Manhattan
Neighborhood                Little Italy
ID              5d8e37f352f64b000873b779
Name                               Zooba
Likes                                 52
Rating                               8.4
Tips                                  16
Name: 0, dtype: object

In [64]:
# finally the lowest rating
egyptian_stats2.iloc[egyptian_stats['Rating'].idxmin()]

Borough                           Queens
Neighborhood                   Ridgewood
ID              550b13c6498e8b5069630cee
Name                        Little Egypt
Likes                                 11
Rating                               8.1
Tips                                   4
Name: 2, dtype: object

In [89]:
# lets visualize our data

rest_map = folium.Map(location=(latitude, longitude), zoom_start=12)

incidents = folium.map.FeatureGroup()


for lat, lng, in egyptian_stats2[['Latitude','Longitude']].values:
    incidents.add_child(
        folium.CircleMarker(
            [lat, lng],
            radius=10, # define how big you want the circle markers to be
            color='yellow',
            fill=True,
            fill_color='blue',
            fill_opacity=0.6
        )
    )
rest_map.add_child(incidents)
